## **Примери: Више-АИ агенти за резервацију хотела**

У данашњем брзом свету, планирање пословног путовања подразумева више од само резервације лета и хотелске собе. Потребан је ниво координације и ефикасности који може бити изазовно постићи. Овде на сцену ступају Више-АИ агенти, који револуционишу начин на који управљамо нашим путовањима.

Замислите тим интелигентних агената на вашем располагању, који заједно раде на томе да сваки аспект вашег путовања буде обрађен са прецизношћу и лакоћом. Уз нашу напредну АИ технологију, креирали смо специјализоване агенте за услуге резервације и организацију итинерера, обезбеђујући беспрекорно и без стреса искуство путовања.

Ово је основни сценарио. Када планирамо пословно путовање, потребно је да се консултујемо са агентом за пословна путовања како бисмо добили информације о авионским картама, хотелима и слично. Кроз АИ агенте, можемо изградити агенте за услуге резервације и агенте за организацију итинерера који ће сарађивати и побољшати ниво интелигенције.


# Иницијализација Azure AI Agent Service и добијање информација о конфигурацији из **.env**

### **.env**

Креирајте .env датотеку

**.env** садржи стринг за повезивање са Azure AI Agent Service, модел који користи AOAI, као и одговарајући Google API Search сервис API, ENDPOINT, итд.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Назив вашег Azure AI Agent Service модела за распоређивање"

[**NOTE**] Биће вам потребан модел са ограничењем од 100,000 токена у минути (Rate Limit) и ограничењем од 600 захтева у минути (Request per minute).

  Модел можете добити у Azure AI Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Стринг за повезивање вашег Azure AI Agent Service пројекта"

  Стринг за повезивање пројекта можете добити у прегледу вашег пројекта на AI Foundry Portal екрану.

- **SERPAPI_SEARCH_API_KEY** = "Ваш SERPAPI Search API КЉУЧ"
- **SERPAPI_SEARCH_ENDPOINT** = "Ваш SERPAPI Search Endpoint"

Да бисте добили Назив модела за распоређивање и Стринг за повезивање пројекта Azure AI Agent Service, потребно је да креирате Azure AI Agent Service. Препоручује се коришћење [овог шаблона](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) за директно креирање. (***Напомена:*** Azure AI Agent Service је тренутно доступан у ограниченим регионима. Препоручује се да се консултујете са [овим линком](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) за подешавање региона.)

Агенту је потребан приступ SERPAPI. Препоручује се регистрација путем [овог линка](https://serpapi.com/searches). Након регистрације, можете добити јединствени API КЉУЧ и ENDPOINT.


# Пријављивање на Azure

Сада треба да се пријавите на Azure. Отворите терминал у VScode и покрените команду `az login`.


# Подешавање

Да бисте покренули овај нотебук, потребно је да инсталирате следеће библиотеке. Ево списка потребних библиотека и одговарајућих команди за инсталацију помоћу pip-а:

azure-identity: За Azure аутентификацију.  
requests: За слање HTTP захтева.  
semantic-kernel: За оквир семантичког језгра (под претпоставком да је ово прилагођена или специфична библиотека, можда ћете морати да је инсталирате са одређеног извора или репозиторијума).  


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Објашњење:  
import asyncio: Ово увози модул asyncio, који пружа подршку за асинхроно програмирање у Пајтону. Омогућава вам да пишете конкурентни код користећи синтаксу async и await.  
from typing: Annotated: Ово увози тип Annotated из модула typing. Annotated се користи за додавање метаподатака у типске назнаке, што може бити корисно за различите сврхе као што су валидација, документација или алати.  


In [ ]:
import asyncio,os
from typing import Annotated

# Објашњење:
Коришћењем `from dotenv import load_dotenv` и `load_dotenv()`, можете лако управљати подешавањима конфигурације и осетљивим информацијама (као што су API кључеви и URL-ови база података) у `.env` датотеци, држећи их одвојено од изворног кода и чинећи вашу апликацију безбеднијом и једноставнијом за конфигурисање.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Објашњење:

Изјава за увоз: from azure.identity.aio import DefaultAzureCredential: Овим се увози класа DefaultAzureCredential из модула azure.identity.aio. Део "aio" у имену модула указује на то да је дизајниран за асинхроне операције.

Сврха DefaultAzureCredential: Класа DefaultAzureCredential је део Azure SDK за Python. Она пружа подразумевани начин за аутентификацију са Azure услугама. Покушава да се аутентификује користећи више метода у одређеном редоследу, као што су променљиве окружења, управљани идентитет и Azure CLI акредитиви.

Асинхроне операције: Модул "aio" указује на то да класа DefaultAzureCredential подржава асинхроне операције. То значи да можете да је користите са asyncio за извршавање захтева за аутентификацију без блокирања.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Објашњење:
Увоз различитих модула и класа из пакета semantic_kernel. Ево детаљног прегледа сваког увоза:

AgentGroupChat из semantic_kernel.agents: Ова класа управља функционалностима везаним за групни чет за AI агенте. AzureAIAgent и AzureAIAgentSettings из semantic_kernel.agents.azure_ai

AzureAIAgent: Ова класа се користи за креирање и управљање AI агентима који користе Azure AI услуге.

AzureAIAgentSettings: Ова класа се користи за конфигурисање подешавања за AzureAIAgent. TerminationStrategy из semantic_kernel.agents.strategies.termination.termination_strategy:

Ова класа дефинише стратегије за прекид извршавања AI агената под одређеним условима. ChatMessageContent из semantic_kernel.contents.chat_message_content:

Ова класа се користи за управљање садржајем чет порука.
AuthorRole из semantic_kernel.contents.utils.author_role:

Ова класа дефинише различите улоге аутора у контексту чет порука.

kernel_function из semantic_kernel.functions.kernel_function_decorator: Овај декоратор се користи за дефинисање kernel функција, које су функције које се могу извршавати у оквиру semantic kernel оквира.
Ови увози постављају неопходне компоненте за креирање и управљање AI агентима који могу да комуницирају у окружењу групног чета, могуће за задатке као што је резервација хотела или сличне активности.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Објашњење:
Следеће увозимо класу CodeInterpreterTool из модула azure.ai.projects.models.

CodeInterpreterTool: Ова класа је део Azure AI SDK-а и користи се за интерпретацију и извршавање кода у контексту AI пројеката. Пружа функционалности за покретање делова кода, анализу кода или интеграцију извршавања кода у оквиру AI радних токова.  
Овај увоз поставља неопходну компоненту за коришћење CodeInterpreterTool-а у вашем пројекту, што може бити корисно за задатке који укључују динамичку интерпретацију и извршавање кода.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Објашњење: 
Класа ApprovalTerminationStrategy пружа специфичну стратегију за прекид рада AI агента. Агент ће прекинути рад ако последња порука у историји интеракције садржи реч "saved". Ово може бити корисно у ситуацијама где се задатак агента сматра завршеним када добије потврду да је нешто "saved". Дефинишите метод интеракције. Након што је план резервације сачуван, може се зауставити када прими сигнал "saved".


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Објашњење:

Овај ред кода иницијализује објекат AzureAIAgentSettings са подразумеваним или унапред дефинисаним подешавањима позивањем методе create(). Овај објекат подешавања (ai_agent_settings) затим се може користити за конфигурисање и управљање AzureAIAgent инстанцом.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Објашњење:
Увозом библиотеке requests, можете лако правити HTTP захтеве и комуницирати са веб сервисима у вашем Python коду.


In [ ]:
import requests

# Објашњење:
Ово је променљива која чува API кључ за приступ SERP (страница резултата претраживача) API услузи. API кључ је јединствени идентификатор који се користи за аутентификацију захтева повезаних са вашим налогом.

'GOOGLE_SEARCH_API_KEY': Ово је привремени текст. Потребно је да замените ''GOOGLE_SEARCH_API_KEY' са вашим стварним SERP API кључем.

Сврха: Сврха ове линије је да се API кључ сачува у променљивој како би се могао користити за аутентификацију захтева ка SERP API услузи. API кључ је неопходан за приступ услузи и извршавање претрага.

Како добити SERP API кључ: Да бисте добили SERP API кључ, следите ове опште кораке на https://serpapi.com (тачни кораци могу варирати у зависности од специфичне SERP API услуге коју користите):

Изаберите SERP API услугу: Постоји неколико доступних SERP API услуга, као што су SerpAPI, Google Custom Search JSON API и друге. Изаберите ону која најбоље одговара вашим потребама.

Региструјте се за налог:

Идите на веб-сајт изабране SERP API услуге https://www.serpapi.com и региструјте се за налог. Можда ће бити потребно да пружите основне информације и потврдите своју имејл адресу.

Креирајте API кључ:

Након регистрације, пријавите се на свој налог и идите на секцију API или контролни панел. Потражите опцију за креирање или генерисање новог API кључа.
Копирајте API кључ:

Када се API кључ генерише, копирајте га. Овај кључ ће се користити за аутентификацију ваших захтева ка SERP API услузи.
Замените привремени текст:

Замените привремени текст у вашем .env фајлу.


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Објашњење:
Класа BookingPlugin пружа методе за резервацију хотела и летова користећи Serpapi.com Google Search API. Она конструише неопходне параметре, шаље API захтеве и обрађује одговоре како би вратила релевантне информације о резервацијама. API кључ (SERPAPI_SEARCH_API_KEY) и крајња тачка (SERPAPI_SEARCH_ENDPOINT) се користе за аутентификацију и слање захтева Google Search API-ју.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Објашњење:
Класа SavePlugin пружа метод saving_plan за чување планова путовања користећи Azure AI услуге. Поставља Azure акредитиве, креира AI агента, обрађује корисничке уносе за генерисање и чување садржаја плана путовања, и управља операцијама чувања датотека и чишћења. Метод враћа "Saved" након успешног завршетка.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Објашњење:
Овај код поставља Azure AI агенте за управљање резервацијама летова и хотела, као и за чување планова путовања на основу уноса корисника. Користи Azure креденцијале за креирање и конфигурисање агената, обрађује уносе корисника кроз групни чет и обезбеђује правилно чишћење након завршетка задатака. Агенти користе специфичне додатке (BookingPlugin и SavePlugin) за извршавање својих одговарајућих задатака.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да превод буде тачан, молимо вас да имате у виду да аутоматизовани преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати меродавним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
